# Introduction to Turing Machines

## Introduction
This section provides a comprehensive introduction to Turing Machines, one of the most fundamental concepts in theoretical computer science. Named after the British mathematician Alan Turing, these abstract machines serve as a mathematical model of computation that, despite their simplicity, can simulate any computer algorithm. Turing Machines are central to our understanding of what problems are computationally solvable. They form the foundation of the Church-Turing thesis, which suggests that any function that can be effectively calculated can be computed by a Turing Machine. This concept has profound implications for computer science, mathematics, and philosophy.

## 1. Definition of Turing Machines

### 1.1 Definition
A Turing machine is a mathematical model of computation that defines an abstract machine, which manipulates symbols on a strip of tape according to a table of rules. Despite its simplicity, the Turing machine can simulate the logic of any computer algorithm.

A Turing machine can be formally defined as a 7-tuple: $M=(Q,\Sigma,\Gamma,T, H, \delta, \triangle)$ where:

* $Q$ is a finite set of states with exactly one START state and zero or more HALT states.
* $\Sigma$ is a finite set of input symbols that can appear in the original input string provided to the machine.
* $\Gamma$ is a finite set of tape symbols the machine can write on the tape, including input symbols plus extra symbols like markers.
* $T$ is a TAPE consisting of a sequence of numbered cells, each holding either a single character or a blank symbol. The input word is written on the tape with one character per cell, starting from the leftmost cell (cell 1). All remaining cells on the tape are initially filled with blanks.
* $H$ is a TAPE HEAD that, in a single step, can read the contents of a cell on the tape, overwrite it with another character, and then move either one cell to the left or right. At the start of computation, the tape head begins by reading the input in the leftmost cell (cell 1).
* $\delta$ is a program or a set of transition functions that tells the machine what to do based on the current state and the symbol under the head.
* $\triangle$ is a blank symbol, which is not part of input symbols or tape symbols.

### 1.2 Example Python Implementation
Below is a Python implementation that simulates the behavior of a Turing Machine.

In [1]:
class TuringMachine:
    def __init__(self, states, input_symbols, tape_symbols, transition_function, blank_symbol):
        """
        Initialize a Turing Machine based on the formal definition M=(Q,Σ,Γ,T,H,δ,△).
        
        Parameters:
        - states: Set of states (Q) with a designated START state and zero or more HALT states
                 Format: {'START', 'q1', 'q2', ..., 'HALT1', 'HALT2', ...}
        - input_symbols: Set of input symbols (Σ)
        - tape_symbols: Set of tape symbols (Γ)
        - transition_function: Dictionary mapping (state, symbol) to (new_state, new_symbol, direction)
                               Direction is 'L' or 'R' for left or right
        - blank_symbol: The blank symbol (△) used for empty cells
        """
        # Ensure states include a START state
        if 'START' not in states:
            raise ValueError("The set of states must include a 'START' state")
            
        # Verify the blank symbol is not in input symbols
        if blank_symbol in input_symbols:
            raise ValueError("The blank symbol cannot be part of the input symbols")
            
        # Initialize the Turing Machine components
        self.states = states
        self.input_symbols = input_symbols
        self.tape_symbols = tape_symbols
        self.transition_function = transition_function
        self.blank_symbol = blank_symbol
        
        # Extract HALT states (states that start with 'HALT')
        self.halt_states = {state for state in states if state.startswith('HALT')}
        
        # Initialize the machine's operational components
        self.tape = []  # T: The tape
        self.head_position = 1  # H: Head position (1-indexed as per definition)
        self.current_state = 'START'  # Current state starts at START
        
    def set_input(self, input_string):
        """
        Set the input on the tape and initialize the machine.
        Input starts at the leftmost cell (cell 1).
        """
        # Validate input symbols
        for symbol in input_string:
            if symbol not in self.input_symbols:
                raise ValueError(f"Invalid input symbol: {symbol}")
        
        # Initialize tape with input starting at position 1 (1-indexed)
        # Python lists are 0-indexed, so we'll add a None at index 0 to align with 1-indexed definition
        self.tape = [None] + list(input_string)
        self.head_position = 1  # Start at leftmost cell (cell 1)
        self.current_state = 'START'  # Reset to START state
    
    def get_current_symbol(self):
        """Read the symbol at the current head position."""
        # If head position is beyond tape length, return blank
        if self.head_position >= len(self.tape):
            return self.blank_symbol
        # If position is valid, return the symbol
        return self.tape[self.head_position]
    
    def write_symbol(self, symbol):
        """Write a symbol at the current head position."""
        # Extend tape if needed
        while self.head_position >= len(self.tape):
            self.tape.append(self.blank_symbol)
        
        # Write the symbol
        self.tape[self.head_position] = symbol
    
    def step(self):
        """
        Execute a single step of the Turing Machine.
        Returns False if halted or no valid transition, True otherwise.
        """
        # If in a HALT state, machine has halted
        if self.current_state in self.halt_states:
            return False
        
        # Get current symbol under head
        current_symbol = self.get_current_symbol()
        
        # Look up transition
        if (self.current_state, current_symbol) in self.transition_function:
            # Get next state, symbol to write, and direction to move
            next_state, new_symbol, direction = self.transition_function[(self.current_state, current_symbol)]
            
            # Update state
            self.current_state = next_state
            
            # Write new symbol
            self.write_symbol(new_symbol)
            
            # Move head
            if direction == 'L':
                # Don't move left beyond cell 1
                self.head_position = max(1, self.head_position - 1)
            elif direction == 'R':
                self.head_position += 1
            else:
                raise ValueError(f"Invalid direction: {direction}. Must be 'L' or 'R'")
            
            return True
        else:
            # No transition defined for current state and symbol
            return False
    
    def run(self, max_steps=10000):
        """
        Run the Turing Machine until it halts or reaches max_steps.
        Returns:
        - 'accepted' if machine reaches a HALT state
        - 'rejected' if no valid transition
        - 'timeout' if max_steps reached
        """
        steps = 0
        
        while steps < max_steps:
            # If in HALT state, machine has accepted
            if self.current_state in self.halt_states:
                return 'accepted'
            
            # Execute a step
            if not self.step():
                # No valid transition
                return 'rejected'
            
            steps += 1
        
        # Max steps reached
        return 'timeout'
    
    def get_tape_contents(self):
        """
        Return the current contents of the tape as a string.
        Excludes the None at index 0 (used for 1-indexing alignment).
        """
        # Convert tape to string, excluding the None at index 0
        # and stopping at the rightmost non-blank symbol
        rightmost = len(self.tape) - 1
        while rightmost > 0 and self.tape[rightmost] == self.blank_symbol:
            rightmost -= 1
        
        return ''.join(self.tape[1:rightmost+1])
    
    def print_configuration(self):
        """Print the current configuration of the machine."""
        # Get tape content (excluding None at index 0)
        tape_content = self.tape[1:] if len(self.tape) > 1 else []
        
        # Extend tape with blanks for visualization if head is beyond current tape
        while len(tape_content) < self.head_position:
            tape_content.append(self.blank_symbol)
        
        # Create string representation of tape
        tape_str = ' '.join(tape_content)
        
        # Create head position indicator
        # Adjust for spaces between symbols and 1-indexing
        head_indicator = ' ' * (2 * (self.head_position - 1)) + '^'
        
        print(f"State: {self.current_state}")
        print(f"Tape:  {tape_str}")
        print(f"Head:  {head_indicator}")

### 1.3 Turing Machine tape
The following image depicts a Turing machine tape. It shows a horizontal tape divided into cells, with each cell numbered sequentially from 1 to 5 across the top. The contents of each cell are clearly marked:

* Cell 1 contains the symbol "a"
* Cell 2 contains the symbol "b"
* Cell 3 contains the symbol "a"
* Cells 4 and 5 contain the blank symbol $\triangle$

An ellipsis "..." indicates that the tape continues indefinitely to the right.

A tape head indicator (shown as an upward-pointing arrow) is positioned beneath Cell 1, pointing to the first "a" symbol. The text "TAPE HEAD" appears below the arrow to clearly label this component. This illustration represents the conceptual model of a Turing machine's memory structure, showing the current configuration of the tape and the position of the read/write head. In Turing machine theory, the tape head can read the current symbol, write a new symbol, and move left or right according to the machine's transition rules.

In [3]:
# from IPython.display import SVG
# SVG('tape_head.svg')
from IPython.display import SVG, display, HTML, Markdown
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

# Load SVG from file
def load_svg_from_file(filename):
    """Load SVG content from a file"""
    with open(filename, 'r', encoding='utf-8') as f:
        svg_content = f.read()
    print(f"SVG loaded from {filename}")
    return svg_content
    
loaded_svg = load_svg_from_file('tape_head.svg')

html_with_caption = f"""
<figure style="text-align: center;">
    {loaded_svg}
    <figcaption style="margin-top: 10px; font-style: italic; color: #666;">
        Figure 1: a Turing machine tape
    </figcaption>
</figure>
"""
display(HTML(html_with_caption))

SVG loaded from tape_head.svg


## 2. Examples of Turing Machines
Let's implement some simple Turing machines to illustrate the concept.

### 2.1 Example 1: A Turing Machine that Accepts $a^nb^n$
#### 2.1.1 Definition:
Using the formal definition structure $M=(Q,\Sigma,\Gamma,T, H, \delta, \triangle)$, we define the Turing machine that accepts the language $L = \{a^nb^n, n \geq 0 \}$ as follows:

* $Q: \{1START, 2, 3, 4, 5, 6HALT\}$, the finite set of states
* $\Sigma = \{a, b\}$, the finite set of input symbols
* $\Gamma = \{a, b, A, B\}$, the finite set of tape symbols (excluding the blank symbol)
* $T$: the tape consisting of cells, with the input starting at cell 1. Initially it contains the input string, with one character per cell. All remaining cells are filled with blank symbols.
* $H$: the tape head that starts at position 1. It can read, write, and move left or right in each step.
* $\delta$: the transition function defined as follows:
    * δ(START, a) = (2, A, R)
    * δ(START, △) = (HALT, △, R)
    * δ(2, a) = (2, a, R)
    * δ(2, B) = (2, B, R)
    * δ(2, b) = (3, B, L)
    * δ(3, B) = (3, B, L)
    * δ(3, a) = (4, a, L)
    * δ(3, A) = (5, A, R)
    * δ(4, a) = (4, a, L)
    * δ(4, A) = (START, A, R)
    * δ(5, B) = (5, B, R)
    * δ(5, △) = (HALT, △, R)
* $△$: the blank symbol, it is not part of the input or tape symbol sets.

#### 2.1.2 Machine Diagram:
The following diagram visualizes the state transitions of this Turing machine. 

```{mermaid}
stateDiagram-v2
    accTitle: State Transitions of a Turing machine
    accDescr: a diagram representing state transitions of a Turing machine
    direction LR
    
    START: 1 START
    q2: 2
    q3: 3
    q4: 4
    q5: 5
    HALT:6  HALT
    
    START --> q2: (a,A,R)
    START --> HALT: (△,△,R)
    
    q2 --> q2: (a,a,R)
    q2 --> q2: (B,B,R)
    q2 --> q3: (b,B,L)
    
    q3 --> q3: (B,B,L)
    q3 --> q4: (a,a,L)
    q3 --> q5: (A,A,R)
    
    q4 --> q4: (a,a,L)
    q4 --> START: (A,A,R)
    
    q5 --> q5: (B,B,R)
    q5 --> HALT: (△,△,R)
    
    classDef start fill:#9f6,stroke:#333,stroke-width:2px
    classDef halt fill:#f96,stroke:#333,stroke-width:2px
    classDef state fill:#bbf,stroke:#333,stroke-width:1px
    
    class START start
    class HALT halt
    class q2,q3,q4,q5 state
```

#### 2.1.3 How this Turing Machine Works:
This Turing machine is designed to recognize strings of the form $a^nb^n$, which means n 'a's followed by exactly n 'b's (where n ≥ 0). Let us explain in detail how this machine processes inputs. The machine works by systematically "matching and marking" 'a's and 'b's. It converts the leftmost 'a' to 'A' (marking it as processed); scans right to find the leftmost 'b' and converts it to 'B' (marking it as processed); returns to the beginning to repeat the process with the next 'a'. When all pairs are matched, it verifies that all input has been processed. If there are equal numbers of 'a's and 'b's, it accepts the input; otherwise, it rejects the input.

Let us examine each state and its role:

* State 1 (START): Processes the leftmost unmarked 'a' or accepts if the input is blank
* State 2: Scans right looking for the leftmost unmarked 'b'
* State 3: After marking 'b', scans left looking for 'a's or 'A's
* State 4: After finding an unmarked 'a', scans left to return to the beginning
* State 5: After finding a marked 'A', verifies all 'b's have been processed

For example, let us process an input: "aabb"

* Start (State 1): Machine reads 'a', replaces it with 'A', moves right, enters State 2. Tape: Aabb
* State 2: Reads 'a', keeps it, moves right, stays in State 2. Tape: Aabb
* State 2 → State 3: Reads 'b', replaces it with 'B', moves left, enters State 3. Tape: AabB
* State 3: Reads 'a', moves left, enters State 4. Tape: AabB
* State 4 → State 1: Reads 'A', moves right, returns to State 1. Tape: AabB
* State 1 → State 2: Reads 'a', replaces it with 'A', moves right, enters State 2. Tape: AAbB
* State 2 → State 3: Reads 'B', keeps it, moves right, stays in State 2. Then reads 'b', replaces it with 'B', moves left, enters State 3. Tape: AABB
* State 3 → State 5: Reads 'B', keeps it, moves left, stays in State 3. Then reads 'A', moves right, enters State 5. Tape: AABB
* State 5 → HALT: Reads 'B', keeps it, moves right, stays in State 5. Then reads blank, moves right, enters HALT state. Input is accepted.

#### 2.1.4 Example Python Implementation
Below is a representative Python implementation of the described Turing machine.

In [4]:
def create_anbn_tm():
    """
    Creates a Turing machine that recognizes strings of the form a^n b^n.
    Based on the state diagram provided in the instructions.
    """
    # Define the components of the Turing machine
    states = {'START', '2', '3', '4', '5', 'HALT'}
    input_symbols = {'a', 'b'}
    tape_symbols = {'a', 'b', 'A', 'B', '△'}  # 'A' marks processed 'a', 'B' marks processed 'b'
    blank_symbol = '△'
    
    # Transition function based on the state diagram
    transition_function = {
        # START state (state 1)
        ('START', 'a'): ('2', 'A', 'R'),  # Replace 'a' with 'A' and move to state 2
        ('START', '△'): ('HALT', '△', 'R'),  # Accept empty string immediately
        
        # State 2
        ('2', 'a'): ('2', 'a', 'R'),      # Skip 'a' and stay in state 2
        ('2', 'B'): ('2', 'B', 'R'),      # Skip 'B' and stay in state 2
        ('2', 'b'): ('3', 'B', 'L'),      # Replace 'b' with 'B' and move to state 3
        
        # State 3
        ('3', 'B'): ('3', 'B', 'L'),      # Skip 'B' and stay in state 3
        ('3', 'a'): ('4', 'a', 'L'),      # Read 'a' and move to state 4
        ('3', 'A'): ('5', 'A', 'R'),      # Read 'A' and move to state 5
        
        # State 4
        ('4', 'A'): ('START', 'A', 'R'),  # Read 'A' and move back to START
        ('4', 'a'): ('4', 'a', 'L'),      # Skip 'a' and stay in state 4
        
        # State 5
        ('5', 'B'): ('5', 'B', 'R'),      # Skip 'B' and stay in state 5
        ('5', '△'): ('HALT', '△', 'R'),   # Read blank and halt
    }
    
    return TuringMachine(states, input_symbols, tape_symbols, transition_function, blank_symbol)

# Test the a^n b^n Turing machine
anbn_tm = create_anbn_tm()

test_strings = ["", "ab", "aabb", "abb", "aaabb", "aaabbb", "bbaa"]

for test_str in test_strings:
    try:
        anbn_tm.set_input(test_str)
        result = anbn_tm.run(1000)
        print(f"Input: '{test_str}', Result: {result}")
        
        # Show final tape contents for debugging
        final_tape = anbn_tm.get_tape_contents()
        print(f"Final tape: {final_tape}")
        print(f"Final state: {anbn_tm.current_state}")
        print("---")
    except ValueError as e:
        print(f"Input: '{test_str}', Error: {e}")
        print("---")

Input: '', Result: accepted
Final tape: 
Final state: HALT
---
Input: 'ab', Result: accepted
Final tape: AB
Final state: HALT
---
Input: 'aabb', Result: accepted
Final tape: AABB
Final state: HALT
---
Input: 'abb', Result: rejected
Final tape: ABb
Final state: 5
---
Input: 'aaabb', Result: rejected
Final tape: AAABB
Final state: 2
---
Input: 'aaabbb', Result: accepted
Final tape: AAABBB
Final state: HALT
---
Input: 'bbaa', Result: rejected
Final tape: bbaa
Final state: START
---


### 2.2 Example 2: A Turing Machine that Accepts PALINDROME
#### 2.2.1 Definition:
We define the Turing machine that accepts the language PALINDROME as follows:

* $Q: \{1START, 2, 3, 4, 5, 6, 7, 8HALT\}$, the finite set of states
* $\Sigma = \{a, b\}$, the finite set of input symbols
* $\Gamma = \{a, b\}$, the finite set of tape symbols (excluding the blank symbol)
* $T$: the tape consisting of cells, with the input starting at cell 1.
* $H$: the tape head that starts at position 1.
* $\delta$: the transition function defined as follows:
    * δ(1_START, a) = (2, △, R)
    * δ(1_START, b) = (5, △, R)
    * δ(1_START, △) = (8_HALT, △, R)
    * δ(2, a) = (2, a, R)
    * δ(2, b) = (2, b, R)
    * δ(2, △) = (3, △, L)
    * δ(3, a) = (4, △, L)
    * δ(3, △) = (8_HALT, △, R)
    * δ(4, a) = (4, a, L)
    * δ(4, b) = (4, b, L)
    * δ(4, △) = (1_START, △, R)
    * δ(5, a) = (5, a, R)
    * δ(5, b) = (5, b, R)
    * δ(5, △) = (6, △, L)
    * δ(6, b) = (7, △, L)
    * δ(6, △) = (8_HALT, △, R)
    * δ(7, a) = (7, a, L)
    * δ(7, b) = (7, b, L)
    * δ(7, △) = (1_START, △, R)
* $△$: the blank symbol, it is not part of the input or tape symbol sets.

#### 2.2.2 Machine Diagram:
The following diagram visualizes the state transitions of this Turing machine. 

```{mermaid}
stateDiagram-v2
    accTitle: State Transitions of a Turing machine
    accDescr: a diagram representing state transitions of a Turing machine
    direction LR
    
    START: 1 START
    q2: 2
    q3: 3
    q4: 4
    q5: 5
    q6: 6
    q7: 7
    HALT: 8 HALT
    
    START --> q2: (a,Δ,R)
    START --> q5: (b,Δ,R)
    START --> HALT: (Δ,Δ,R)
    
    q2 --> q2: (a,a,R)
    q2 --> q2: (b,b,R)
    q2 --> q3: (Δ,Δ,L)
    
    q3 --> q4: (a,Δ,L)
    q3 --> HALT: (Δ,Δ,R)
    
    q4 --> q4: (a,a,L)
    q4 --> q4: (b,b,L)
    q4 --> START: (Δ,Δ,R)
    
    q5 --> q5: (a,a,R)
    q5 --> q5: (b,b,R)
    q5 --> q6: (Δ,Δ,L)
    
    q6 --> q7: (b,Δ,L)
    q6 --> HALT: (Δ,Δ,R)
    
    q7 --> q7: (a,a,L)
    q7 --> q7: (b,b,L)
    q7 --> START: (Δ,Δ,R)
    
    classDef start fill:#9f6,stroke:#333,stroke-width:2px
    classDef halt fill:#f96,stroke:#333,stroke-width:2px
    classDef state fill:#bbf,stroke:#333,stroke-width:1px
    
    class START start
    class HALT halt
    class q2,q3,q4,q5,q6,q7 state
```

#### 2.2.3 How this Turing Machine Works:
A palindrome is a string that reads the same forward and backward, like "aba" or "abba". Let us explain in detail how this machine processes inputs. The machine uses a systematic approach of "matching and erasing" from the outside inward. It examines the leftmost character and travels to the rightmost character to check if it's a match. If they match, erases both and repeats the process; if they don't match or the machine encounters an unexpected pattern, it rejects the input. If it successfully processes the entire string (reaching the middle with all matches), it accepts the input.

Two Processing Paths:

* Upper path (States 2-3-4) for processing 'a' characters
* Lower path (States 5-6-7) for processing 'b' characters

Let us examine each state and its role:

* State 1 (START): Examines the first character and directs processing based on whether it's 'a', 'b', or blank
* State 2: After seeing 'a' at the start, scans right until reaching the end of the string
* State 3: Looks for a matching 'a' at the end of the string
* State 4: After finding and erasing the matching 'a', returns to the start for the next iteration
* State 5: After seeing 'b' at the start, scans right until reaching the end of the string
* State 6: Looks for a matching 'b' at the end of the string
* State 7: After finding and erasing the matching 'b', returns to the start for the next iteration
* State 8 (HALT): Accepts the input as a palindrome

For example, let us process an input: "aba"

* Start (State 1):     * Machine reads 'a', erases it (writes blank Δ), moves right, and enters State 2. Tape: Δba
* State 2: Reads 'b', keeps it, moves right, stays in State 2. Tape: Δba
* State 2 (continued): Reads 'a', keeps it, moves right, stays in State 2. Tape: Δba
* State 2 → State 3: Reaches the end (reads blank), moves left, enters State 3. Tape: Δba
* State 3 → State 4: Reads 'a', erases it (writes blank), moves left, enters State 4. Tape: ΔbΔ
* State 4 → State 1: Reads 'b', keeps it, moves left, stays in State 4. Then reads blank, moves right, returns to State 1. Tape: ΔbΔ
* State 1 → State 5: Reads 'b', erases it (writes blank), moves right, enters State 5. Tape: ΔΔΔ
* State 5 → State 6: Reads blank, moves left, enters State 6. Tape: ΔΔΔ
* State 6 → State 8: Reads blank, moves right, enters State 8 (HALT). Input accepted as a palindrome

#### 2.2.4 Example Python Implementation
Below, we'll demonstrate how to implement this Turing machine.

In [5]:
def create_palindrome_tm():
    """
    Creates a Turing machine that recognizes palindromes based on the provided state diagram.
    The machine accepts strings that read the same forward and backward.
    """
    # Define the components of the Turing machine
    states = {'1_START', '2', '3', '4', '5', '6', '7', '8_HALT'}
    input_symbols = {'a', 'b'}
    tape_symbols = {'a', 'b', '△'}  # '_' is the blank symbol (Δ)
    blank_symbol = '△'
    
    # Transition function based on the state diagram
    # (current_state, current_symbol) -> (next_state, write_symbol, move_direction)
    transition_function = {
        # From START (state 1)
        ('1_START', 'a'): ('2', '△', 'R'),       # (a,Δ,R) to state 2
        ('1_START', 'b'): ('5', '△', 'R'),       # (b,Δ,R) to state 5
        ('1_START', '△'): ('8_HALT', '△', 'R'),  # (Δ,Δ,R) to HALT - empty string is a palindrome
        
        # State 2 - Processing 'a' at the start
        ('2', 'a'): ('2', 'a', 'R'),             # (a,a,R) self-loop
        ('2', 'b'): ('2', 'b', 'R'),             # (b,b,R) self-loop
        ('2', '△'): ('3', '△', 'L'),             # (Δ,Δ,L) to state 3
        
        # State 3 - Moving left to find matching 'a'
        ('3', 'a'): ('4', '△', 'L'),             # (a,Δ,L) to state 4
        ('3', '△'): ('8_HALT', '△', 'R'),        # (Δ,Δ,R) to HALT
        
        # State 4 - Processing after finding matching 'a'
        ('4', 'a'): ('4', 'a', 'L'),             # (a,a,L) self-loop
        ('4', 'b'): ('4', 'b', 'L'),             # (b,b,L) self-loop
        ('4', '△'): ('1_START', '△', 'R'),       # (Δ,Δ,R) back to START
        
        # State 5 - Processing 'b' at the start
        ('5', 'a'): ('5', 'a', 'R'),             # (a,a,R) self-loop
        ('5', 'b'): ('5', 'b', 'R'),             # (b,b,R) self-loop
        ('5', '△'): ('6', '△', 'L'),             # (Δ,Δ,L) to state 6
        
        # State 6 - Moving left to find matching 'b'
        ('6', 'b'): ('7', '△', 'L'),             # (b,Δ,L) to state 7
        ('6', '△'): ('8_HALT', '△', 'R'),        # (Δ,Δ,R) to HALT
        
        # State 7 - Processing after finding matching 'b'
        ('7', 'a'): ('7', 'a', 'L'),             # (a,a,L) self-loop
        ('7', 'b'): ('7', 'b', 'L'),             # (b,b,L) self-loop
        ('7', '△'): ('1_START', '△', 'R'),       # (Δ,Δ,R) back to START
    }
    
    return TuringMachine(states, input_symbols, tape_symbols, transition_function, blank_symbol)

# Test the palindrome Turing machine
def test_palindrome_tm():
    palindrome_tm = create_palindrome_tm()
    
    test_strings = ["", "a", "b", "aa", "bb", "aba", "bab", "abba", "baab", "abb", "bba", "abab"]
    
    print("Testing Palindrome Turing Machine:")
    print("----------------------------------")
    
    for test_str in test_strings:
        try:
            palindrome_tm.set_input(test_str)
            result = palindrome_tm.run(1000)  # Set a high step limit
            
            print(f"Input: '{test_str}'")
            print(f"Result: {result}")
            print(f"Final state: {palindrome_tm.current_state}")
            print(f"Final tape: {palindrome_tm.get_tape_contents()}")
            print("----------------------------------")
        except Exception as e:
            print(f"Input: '{test_str}', Error: {e}")
            print("----------------------------------")

# Define the TuringMachine class to match our implementation
class TuringMachine:
    def __init__(self, states, input_symbols, tape_symbols, transition_function, blank_symbol):
        """
        Initialize a Turing Machine with the 7-tuple definition.
        
        Parameters:
        - states: Set of states with a designated START state and HALT states
        - input_symbols: Set of input symbols
        - tape_symbols: Set of tape symbols
        - transition_function: Dictionary mapping (state, symbol) to (new_state, new_symbol, direction)
        - blank_symbol: The blank symbol used for empty cells
        """
        self.states = states
        self.input_symbols = input_symbols
        self.tape_symbols = tape_symbols
        self.transition_function = transition_function
        self.blank_symbol = blank_symbol
        
        # Initialize the machine's state
        self.tape = [None]  # Using 1-indexing, with None at position 0
        self.head_position = 1  # Start at position 1
        self.current_state = '1_START'  # Start state
        
    def set_input(self, input_string):
        """Set the input on the tape and initialize the machine."""
        # Validate input symbols
        for symbol in input_string:
            if symbol not in self.input_symbols:
                raise ValueError(f"Invalid input symbol: {symbol}")
        
        # Initialize tape with input starting at position 1
        self.tape = [None] + list(input_string)
        self.head_position = 1
        self.current_state = '1_START'
        
    def get_current_symbol(self):
        """Read the symbol at the current head position."""
        if self.head_position >= len(self.tape):
            return self.blank_symbol
        return self.tape[self.head_position]
    
    def write_symbol(self, symbol):
        """Write a symbol at the current head position."""
        # Extend tape if needed
        while self.head_position >= len(self.tape):
            self.tape.append(self.blank_symbol)
        
        self.tape[self.head_position] = symbol
    
    def step(self):
        """
        Execute a single step of the Turing Machine.
        Returns False if halted or no valid transition, True otherwise.
        """
        # Check if in a HALT state
        if '8_HALT' in self.current_state:
            return False
        
        # Get current symbol
        current_symbol = self.get_current_symbol()
        
        # Check for valid transition
        if (self.current_state, current_symbol) in self.transition_function:
            next_state, new_symbol, direction = self.transition_function[(self.current_state, current_symbol)]
            
            # Update state
            self.current_state = next_state
            
            # Write the new symbol
            self.write_symbol(new_symbol)
            
            # Move head
            if direction == 'L':
                self.head_position = max(1, self.head_position - 1)
            elif direction == 'R':
                self.head_position += 1
            
            return True
        else:
            # No valid transition
            return False
    
    def run(self, max_steps=1000):
        """Run the Turing Machine until it halts or reaches max_steps."""
        steps = 0
        
        while steps < max_steps:
            # If in HALT state, machine has accepted
            if '8_HALT' in self.current_state:
                return 'accepted'
            
            # Execute a step
            if not self.step():
                return 'rejected'  # No valid transition
            
            steps += 1
        
        return 'timeout'  # Exceeded maximum steps
    
    def get_tape_contents(self):
        """Return the non-blank contents of the tape."""
        # Skip the None at index 0 and exclude trailing blanks
        if len(self.tape) <= 1:
            return ""
            
        content = self.tape[1:]
        
        # Remove trailing blanks
        while content and content[-1] == self.blank_symbol:
            content.pop()
            
        return ''.join([str(symbol) for symbol in content if symbol != self.blank_symbol])
    
    def print_configuration(self):
        """Print the current configuration of the machine."""
        tape_str = ' '.join([str(symbol) if symbol is not None and symbol != self.blank_symbol else 'Δ' 
                           for symbol in self.tape[1:]])
        
        head_indicator = ' ' * (2 * (self.head_position - 1)) + '^'
        
        print(f"State: {self.current_state}")
        print(f"Tape:  {tape_str}")
        print(f"Head:  {head_indicator}")

# Run the test if this file is executed directly
if __name__ == "__main__":
    test_palindrome_tm()

Testing Palindrome Turing Machine:
----------------------------------
Input: ''
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'a'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'b'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'aa'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'bb'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'aba'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'bab'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'abba'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'baab'
Result: accepted
Final state: 8_HALT
Final tape: 
----------------------------------
Input: 'abb'
Result: rejected
Final state: 3
Final tape

## 3. Processing Input with Turing Machines
A Turing machine processes input quite differently from a finite automaton or a Pushdown automaton, despite their similarities. This section explains the operation of a Turing machine when processing input and details how the transition function drives this process.

### 3.1 One-Way vs. Two-Way Infinite Tapes
It’s worth noting that in Alan Turing’s original 1936 paper, "On Computable Numbers, with an Application to the Entscheidungsproblem," the Turing machine was defined with a tape that stretched infinitely in both directions. In contrast, the model used in this textbook simplifies things by using a tape that’s only infinite to the right. This change introduces a boundary at the first cell, something we’ll discuss below. Even though the tape setup is different, it turns out this doesn’t change what the machine can actually do. Turing machines with one-way infinite tapes are just as powerful as those with two-way infinite tapes. Anything one can compute, the other can too. You just need a bit of extra bookkeeping to manage the boundary on the left side. This equivalence is important because it shows that the fundamental power of Turing machines doesn’t depend on the direction the tape goes. The core idea of what it means for something to be "computable" stays the same.

### 3.2 How a Turing Machine Processes Input
Let's start by understanding what happens when a Turing machine runs on an input string: 

* Initial Configuration:
    * The input string is placed on the tape, one symbol per cell, starting at position 1
    * All other cells are filled with the blank symbol ($\triangle$)
    * The tape head starts at position 1 (the leftmost cell of the input)
    * The machine begins in the START state
* Execution Process:
    * In each step, the machine reads the symbol under the tape head
    * Based on the current state and the read symbol, the machine:
        * Transitions to a new state or remains in the current state
        * Writes a symbol on the current cell (possibly the same symbol)
        * Moves the tape head one position left or right
    * This process continues until the machine reaches a HALT state or has no valid transition
* Acceptance:
    * If the machine reaches a HALT state, the input is accepted
    * If the machine gets stuck (no valid transition exists for the current state and symbol), the input is rejected
    * If the machine runs forever without halting, the input is neither accepted nor rejected
* Important Boundary Condition:
    * If the TAPE HEAD attempts to move LEFT from cell 1, the machine crashes and immediately rejects the input
    * This occurs regardless of the state the machine transitions to, even if it’s a HALT state.
    * This boundary condition exists because the tape is only infinite to the right, not to the left
    * In our implementation, we've enforced this by ensuring the head position never goes below 1

### 3.3 The Transition Function in Detail
The transition function $\delta$ is the heart of a Turing machine. It dictates the machine's behavior for every possible situation it might encounter. Formally, the transition function maps: $\delta(Q × \Gamma) = Q × \Gamma × \{L, R\}$ where:

* $Q$ is the set of states
* $\Gamma$ is the set of tape symbols
* $L$ and $R$ represent left and right movements of the TAPE HEAD
* The $×$  symbol indicates a Cartesian product, $Q × \Gamma$ meaning the function takes as input a pair consisting of a state and a tape symbol

For each combination of a state $q_1 \in Q$ and a tape symbol $a_1 \in \Gamma$, the transition function specifies:

* The next state $q_2 \in Q$
* The symbol $a_2 \in \Gamma$ to write on the current cell
* The direction $d \in \{L, R\}$ to move the TAPE HEAD

For example, the transition function $\delta(START, a) = (HALT, b, R)$ means that when the machine is in the START state and reads the symbol $a$, it transitions to the HALT state, writes $b$ on the tape, and moves the TAPE HEAD one cell to the right.

In our Python implementation, this is represented as a dictionary where:

* The key is a tuple (current_state, current_symbol)
* The value is a tuple (next_state, symbol_to_write, direction_to_move)



## 4. Turing Machines for Regular Languages
Regular languages are a subset of formal languages that can be recognized by finite automata (FA). Regular languages form a subset of formal languages that can be recognized by finite automata (FA). In the following subsection, we demonstrate that any regular language can also be recognized by a Turing machine (TM). This is achieved by systematically converting a finite automaton into an equivalent Turing machine. This conversion highlights that Turing machines are more powerful than finite automata when it comes to defining or recognizing formal languages.

The process of converting a finite automaton to a Turing machine is straightforward:

* Start with an FA that accepts the regular language $L$
* Transform edge labels: Change each transition label $a$ to $(a, a, R)$, meaning "read 'a', write 'a', move right"
* Rename the start state: Change the initial state (typically labeled with a "-" symbol in a FA) to "START"
* Handle acceptance: Remove the accepting state markers (typically "+" symbols in a FA), and instead add transitions from each accepting state to a new "HALT" state with edge labels $(\triangle, \triangle, R)$, where $\triangle$ represents the blank symbol.

This conversion works because the Turing machine simulates the FA by:

* Reading the input string and moving right, just like an FA would
* Following the exact same state transitions as the FA
* When reaching the end of the input (indicated by a blank symbol):
    * If the current state corresponds to an accepting state in the FA, the TM will transition to HALT and accept the input;
    * If the current state is not an accepting state in the FA, the TM will have no valid transition for the blank symbol and will reject the input.

The resulting TM accepts exactly the same strings as the original FA.

### 4.1 Example of Converting a FA to a TM
Consider the following FA, which accepts all strings containing a double occurrence of the letter $a$ (i.e., the substring $aa$).

```{mermaid}
flowchart LR
    accTitle: Example of Converting a FA to a TM
    accDescr: a diagram representing Converting a FA to a TM
    q0((q0-))
    q1((q1))
    q2((q2+))
    
    %% Self loop on q0
    q0 -->|b| q0
    
    %% Transitions between q0 and q1
    q0 -->|a| q1
    q1 -->|b| q0
    
    %% Transition from q1 to q2
    q1 -->|a| q2
    
    %% Self loop on q2
    q2 -->|a,b| q2
    
    %% Style to match the image
    %% Position q0 on the left
    %% Position q1 in the middle
    %% Position q2 on the right
    classDef default fill:#fff,stroke:#333,stroke-width:1px
    classDef start fill:#fff,stroke:#333,stroke-width:1px
    classDef accept fill:#fff,stroke:#333,stroke-width:3px
    
    class q0 start
    class q2 accept
```

It can be converted into an equivalent Turing Machine, as illustrated below:

```{mermaid}
flowchart LR
    accTitle: Example of Converting a FA to a TM
    accDescr: a diagram representing a TM converted from a FA
    start((1 START))
    state2((2))
    halt[HALT 3]
    
    %% Self loop on start state
    start -->|"(b, b, R)"| start
    
    %% Transition from start to state2
    start -->|"(a, a, R)"| state2
    
    %% Transition from state2 to halt
    state2 -->|"(a, a, R)"| halt
    
    %% Transition from state2 back to start
    state2 -->|"(b, b, R)"| start
    
    %% Style to match the image
    classDef default fill:#fff,stroke:#333,stroke-width:1px
    classDef halt fill:#fff,stroke:#333,stroke-width:1px,rx:15,ry:15
    
    class halt halt
```

Note that the HALT state of the Turing Machine does not require looping transitions, as reaching this state signifies that the machine stops processing further input and accepts the string. This conversion technique demonstrates a fundamental relationship between finite automata and Turing machines: Turing machines are strictly more powerful than finite automata, capable of recognizing all regular languages plus many more complex languages that finite automata cannot handle.

## 5. Three Classes of Input Strings to Turing Machines
### 5.1 Definition
When a Turing machine processes an input string, exactly one of three outcomes must occur:

* The machine accepts the input
* The machine rejects the input
* The machine loops infinitely

These three outcomes define three mutually exclusive and exhaustive classes for all possible input strings to any Turing machine. For any Turing machine $T$ and input string $s$, we can classify $s$ into exactly one of three classes:

1. Accept class: The set of strings that $T$ accepts, denoted as $ACCEPT(T)$
2. Reject class: The set of strings that $T$ explicitly rejects after a finite number of steps, denoted as $REJECT(T)$
3. Loop class: The set of strings on which $T$ runs forever without halting, denoted as $LOOP(T)$

### 5.2 Examples

#### 5.2.1 Example 1. Turning Machine that Accepts all strings with $aa$
In the previous section, we discussed how to convert a finite automaton (FA) into a Turing machine (TM), using the language of all strings containing the substring $aa$ as an example. Now, let's modify that TM by adding a new loop transition $\delta(START, △) = (START, △, R)$ to the START state. The updated machine is visualized as follows.

```{mermaid}
flowchart LR
    accTitle: A modified TM
    accDescr: a diagram representing a Modified TM by adding a new loop transition
    start((1 START))
    state2((2))
    halt[HALT 3]
    
    %% Self loop on start state
    start -->|"(b, b, R)"| start
    start -->|"(△, △, R)"| start
    
    %% Transition from start to state2
    start -->|"(a, a, R)"| state2
    
    %% Transition from state2 to halt
    state2 -->|"(a, a, R)"| halt
    
    %% Transition from state2 back to start
    state2 -->|"(b, b, R)"| start
    
    %% Style to match the image
    classDef default fill:#fff,stroke:#333,stroke-width:1px
    classDef halt fill:#fff,stroke:#333,stroke-width:1px,rx:15,ry:15
    
    class halt halt
```

This modified TM still accepts the same language. However, there is an important behavioral difference:
In the original machine, if the input ended while the TM was in the START state (i.e., it encountered a blank symbol), there was no defined transition, causing the machine to implicitly reject the input by halting without accepting.
In the modified version, the new loop transition allows the machine to stay in the START state, write a blank symbol, and move right upon reading a blank. This prevents an immediate rejection and subtly changes how the machine handles end-of-input situations. However, this change also introduces the possibility of infinite looping: if the machine reaches the end of the input in the START state (meaning it has not seen "aa" in the input), it will continue moving right forever, never halting. Thus, some input strings that would have been rejected before will now cause the machine to loop indefinitely instead of halting with a rejection. 

The following table categorizes all possible inputs to this Turing machine:

| Input Class | Input Type | Behavior |
|------------|------------|----------|
| Accept Class | Strings containing "aa" | The machine finds the pattern "aa", transitions to state q2, and eventually reaches the HALT state. These strings are accepted in a finite number of steps. Examples: "aa", "aab", "baa", "aaaa". |
| Reject Class | Strings that end with "a" but don't contain "aa" | When processing strings ending with "a" (but not containing "aa"), the machine will be in state q2 when it reads the final symbol "a". Since q2 does not have a transition on blank sybmol, the machine will crash and implicitly rejects the input. Examples: "a", "ba", "baba". |
| Loop Class | Strings not containing "aa" and not ending with "a" | The machine never encounters the pattern "aa" and isn't in state q2 when it reaches the end. When it reaches the end of input (blank symbol) in the START state, it continues moving right indefinitely due to the $\delta(START, △) = (START, △, R)$ transition. The machine never halts. Examples: "" (empty string), "b", "bb", "babab". |

#### 5.2.2 Example 2
The following Turing machine recognizes strings over the alphabet $\{a, b, c\}$ that contain an even number of 'a's (including zero 'a's) and no 'c'. The machine works by keeping track of parity: whether it has seen an even or odd number of 'a's so far. The START state of the machine is also called q_even, reflecting that we begin having seen 0 'a's (which is an even number).

```{mermaid}
stateDiagram-v2
    accTitle: A TM Accepting Strings containing an even number of 'a's and no 'c'
    accDescr: a diagram representing a TM that accepts Strings containing an even number of 'a's and no 'c'
    direction LR
    START(q_even)
    
    START(q_even) --> q_odd: (a, a, R)
    START(q_even) --> START(q_even): (b, b, R)
    START(q_even) --> HALT: (△, △, R)
    START(q_even) --> q_loop: (c, c, R)
    
    q_odd --> START(q_even): (a, a, R)
    q_odd --> q_odd: (b, b, R)    
    q_odd --> q_loop: (c, c, R)    

    q_loop --> q_loop: (a, a, R)
    q_loop --> q_loop: (b, b, R)
    q_loop --> q_loop: (c, c, L)
    q_loop --> q_loop: (△, △, L)
    
```

How does this machine work:

* The machine starts in START(or q_even), indicating we've seen 0 'a's so far (which is even)
* For inputs without 'c':
    * For each 'a' encountered, it toggles between q_even and q_odd states
    * Each 'b' is simply skipped (doesn't affect the parity count)
* When it reaches the end of the input (blank symbol):
    * If in state q_even, it transitions to HALT (even number of 'a's)
    * If in state q_odd, there is no defined transition, so the machine implicitly crashes and rejects the input
* For inputs containing 'c':
    * When the machine encounters 'c', it transitions to the q_loop state
    * In q_loop state, on inputs 'a' or 'b', it keeps moving right
    * In q_loop state, on input 'c' or blank, it moves left. This creates a "back-and-forth" pattern that never terminates

The following table categorizes all possible inputs to this Turing machine:

| Input Class | Input Type | Behavior |
|-------------|------------|----------|
| **Accept Class** | Strings with an even number of 'a's and no 'c's:<br>- Empty string ("") <br>- Strings with only 'b's ("b", "bb", etc.)<br>- Strings with 2, 4, 6... 'a's ("aa", "aabb", "baababa", etc.) | 1. Machine processes the input from left to right<br>2. Ends in state q_even when it reaches the blank symbol<br>3. Transitions to HALT state<br>4. Accepts the input |
| **Reject Class** | Strings with an odd number of 'a's and no 'c's:<br>- Strings with 1 'a' ("a", "ab", "ba", etc.)<br>- Strings with 3, 5, 7... 'a's ("aaa", "aabaaba", etc.) | 1. Machine processes the input from left to right<br>2. Ends in state q_odd when it reaches the blank symbol<br>3. Has no valid transition for (q_odd, △)<br>4. Implicitly rejects by halting without accepting |
| **Loop Class** | Strings containing at least one 'c':<br>- "c"<br>- "ac", "bc", "ca", "cb"<br>- Any string with 'c' anywhere, regardless of the number of 'a's | 1. Machine processes input until it encounters 'c'<br>2. Transitions to q_loop state<br>3. When it eventually reaches another 'c' or a blank, it moves left<br>4. Continues moving back and forth between symbols indefinitely<br>5. Never halts |

### 5.3 Why the LOOP Class is Essential in Turing Machine Theory
The LOOP class is a crucial concept in the theory of computation for several important reasons:

1. Completeness of Classification: The LOOP class completes the classification system for Turing machines. Without this class, we would have no way to categorize inputs that cause a machine to run forever. Every input to a Turing machine must either be accepted, rejected, or cause the machine to loop indefinitely - these three classes together form a complete partitioning of all possible inputs.
2. Undecidability and the Halting Problem: The existence of the LOOP class is directly connected to one of the most fundamental results in computer science: the Halting Problem. The Halting Problem shows that it is impossible to create an algorithm that can determine, for every possible program and input, whether that program will eventually halt or run forever.
The LOOP class represents precisely those inputs for which a machine will never halt.
3. Differentiating Between Types of Languages: The LOOP class helps us distinguish between different types of languages:
* Decidable languages: A language is decidable if there exists a Turing machine that
    * accepts all strings in the language,
    * rejects all strings not in the language, and
    * has an empty LOOP class. These are the most well-behaved languages.
* Recognizable languages: A language is recognizable (but not decidable) if there exists a Turing machine that
    * accepts all strings in the language,
    * reject or loop forever for strings not in the language.
    * in this case, the LOOP class is non-empty.
4. Theoretical Limitations of Computing: The LOOP class represents a fundamental limitation of computation. It shows that there are problems for which an algorithm can be specified, but it will never terminate for certain inputs. This has profound implications:
    * Not all well-defined problems are effectively computable
    * Some computational processes cannot be predicted without actually running them
    * There are inherent limits to what can be calculated algorithmically

### 5.4 Practical Implications: 
In our example Turing machine that recognizes strings containing "aa" but loops on strings without "aa", we saw how a small modification (adding a loop transition on blank symbols) dramatically changed the behavior. This illustrates an important practical concern in computing:

* Programs can contain infinite loops that might be difficult to detect
* Determining whether a program will terminate for all inputs is generally impossible
* Software verification and testing must contend with the possibility of non-termination

The LOOP class isn't merely a theoretical curiosity: it represents a fundamental aspect of computation that affects both the theoretical foundations of computer science and practical aspects of software development.


## 6. Practice Exercises
### 6.1 Exercise 1: Turing Machine State Diagram
Draw a state diagram for a Turing machine that accepts strings over $\{a, b\}$ that:

* End with the letter 'a'
* Contain exactly one 'b'
* Have an odd number of 'b's

### 6.2 Exercise 2: Machine Execution Tracing
For the Turing machine that accepts strings with an even number of 'a's and no 'c', trace through the execution steps for the following inputs:

* "aba"
* "baa"
* "abba"

Show the state, tape contents, and head position at each step.

### 6.3 Exercise 3: Input Classification
For the following Turing machine transition function, classify each of the given strings as Accept, Reject, or Loop:

* states = {q0, q1, qaccept}
* input_symbols = {0, 1}
* tape_symbols = {0, 1, _}
* initial_state = q0
* accept_states = {qaccept}

Transitions:

* (q0, 0) -> (q0, 0, R)
* (q0, 1) -> (q1, 1, R)
* (q0, _) -> (qaccept, _, R)
* (q1, 0) -> (q1, 0, R)
* (q1, 1) -> (q0, 1, R)
* (q1, _) -> (q1, _, L)

Inputs:
* "01"
* "00"
* "101"
* "111"
* ""

### 6.4 Exercise 4: Converting FA to TM
Convert the following finite automaton to a Turing machine:

* States: {q0, q1, q2}
* Input alphabet: {a, b}
* Start state: q0
* Accept states: {q2}

Transitions:

* q0 on 'a' → q1
* q0 on 'b' → q0
* q1 on 'a' → q1
* q1 on 'b' → q2
* q2 on 'a' → q1
* q2 on 'b' → q0

Show your transition function for the resulting Turing machine.

### 6.5 Exercise 5: Modifying a TM
Take the Turing machine that accepts strings with an even number of 'a's and no 'c', modify it by removing the transition between q_odd and q_loop, and analyze how this change affects the machine’s behavior with respect to the three classes of input: ACCEPT, REJECT, and LOOP.

### 6.6 Exercise 6: Implement in Python
Implement the following Turing machines in Python, using the TuringMachine class from the notebook:

* A machine that increments a binary number (e.g., "1011" becomes "1100")
* A machine that recognizes palindromes over {0, 1}
* A machine that accepts strings where the number of 'a's equals the number of 'b's

Test each implementation with appropriate inputs.


## 7. Further Reading
* "Introduction to the Theory of Computation" by Michael Sipser, Section 3.1
* "Introduction to Computer Theory" by Daniel I.A. Cohen, Chapter 19
* "Automata Theory, Languages, and Computation" by Hopcroft, Motwani, and Ullman, Chapter 8